<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Amazon_Domain_Full_Dataset_Pre_training_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This is the full amazon domain pre-training of the roberta-base model (NO ADAPTERS).  It uses the 25M reviews for 1 pass of 12.5k steps

## ERRORS OUT DUE TO LACK OF MEMORY

# To Do

*   Check for the correct evaluation metric from the paper
*   Include Perplexity?

# Questions

*   For Training Complete Domain Setting, should we use the condensed model (putting as many reviews in the token window) to expedite training?  I think yes, just use 25M reviews and I think the training is roughly equivelant.
*   What is the correct domain training metric?
*   Need to also create Non-Domain Pretrained Model
*   Batch size in paper for Domain training is 2058.  However, we are condensing the reviews to maximize the 512 block size.  I calculated an average review size of 86, so I used a condensed block has ~6 reviews(512/86). A batch size of 346 has roughly the same number of reviews per step.  Is this correct?




# Setup

In [1]:
!pip install datasets
!pip install transformers[torch]
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━

This is the full 25 million reviews dataset "BigTMiami/amazon_25M_reviews_condensed".  

In [2]:
from datasets import load_dataset

dataset = load_dataset("BigTMiami/amazon_25M_reviews_condensed")
dataset

Resolving data files:   0%|          | 0/58 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/58 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4309656 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8360 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/8453 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/59 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4309656
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8360
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8453
    })
})

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
block_size = tokenizer.model_max_length
print(f"block_size:{block_size}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

block_size:512


# Train

In [4]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_config(config)

In [5]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="amazon_domain_pretrained_model",
    learning_rate=0.0005, # Paper for DAPT training
    per_device_train_batch_size=346,
    per_device_eval_batch_size=346,
    num_train_epochs=1, # 1 pass, 12k steps, 25 million reviews
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [6]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [7]:
import math
eval_results = trainer.evaluate()
eval_results
print(f"Before Training")
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 33.17 GiB. GPU 0 has a total capacity of 39.56 GiB of which 4.87 GiB is free. Process 55465 has 34.69 GiB memory in use. Of the allocated memory 34.16 GiB is allocated by PyTorch, and 40.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.05 GiB. GPU 0 has a total capacity of 39.56 GiB of which 1.18 GiB is free. Process 55465 has 38.37 GiB memory in use. Of the allocated memory 37.85 GiB is allocated by PyTorch, and 32.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
eval_results = trainer.evaluate()
eval_results
print(f"After Training")
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()